# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 12:22PM,246165,10,8515458,"Snap Medical Industries, LLC",Released
1,Sep 12 2022 12:22PM,246165,10,8515459,"Snap Medical Industries, LLC",Released
2,Sep 12 2022 12:22PM,246165,10,8515460,"Snap Medical Industries, LLC",Released
3,Sep 12 2022 12:22PM,246165,10,8515461,"Snap Medical Industries, LLC",Released
4,Sep 12 2022 12:22PM,246165,10,8515462,"Snap Medical Industries, LLC",Released
5,Sep 12 2022 12:22PM,246165,10,8515463,"Snap Medical Industries, LLC",Released
6,Sep 12 2022 12:22PM,246165,10,8515464,"Snap Medical Industries, LLC",Released
7,Sep 12 2022 12:22PM,246165,10,8516588,"Snap Medical Industries, LLC",Released
8,Sep 12 2022 12:22PM,246165,10,8516589,"Snap Medical Industries, LLC",Released
9,Sep 12 2022 12:22PM,246165,10,8516590,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,246160,Released,6
23,246161,Released,20
24,246163,Released,14
25,246164,Released,1
26,246165,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246160,NaN,NaN,6.0
246161,NaN,NaN,20.0
246163,NaN,NaN,14.0
246164,NaN,NaN,1.0
246165,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246060,0.0,0.0,1.0
246106,0.0,0.0,1.0
246109,2.0,14.0,1.0
246124,0.0,0.0,3.0
246126,0.0,22.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246060,0,0,1
246106,0,0,1
246109,2,14,1
246124,0,0,3
246126,0,22,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246060,0,0,1
1,246106,0,0,1
2,246109,2,14,1
3,246124,0,0,3
4,246126,0,22,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246060,,,1
1,246106,,,1
2,246109,2,14,1
3,246124,,,3
4,246126,,22,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 12:22PM,246165,10,"Snap Medical Industries, LLC"
14,Sep 12 2022 12:17PM,246164,19,"GUSA Granules USA, Inc."
15,Sep 12 2022 12:15PM,246163,10,"Methapharm, Inc."
29,Sep 12 2022 12:10PM,246161,19,"Innogenix, LLC"
49,Sep 12 2022 12:08PM,246160,10,Emerginnova
55,Sep 12 2022 12:04PM,246157,19,"GUSA Granules USA, Inc."
56,Sep 12 2022 11:55AM,246156,16,Sartorius Stedim North America
61,Sep 12 2022 11:55AM,246155,19,"AdvaGen Pharma, Ltd"
62,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc."
83,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 12 2022 12:22PM,246165,10,"Snap Medical Industries, LLC",,,14
1,Sep 12 2022 12:17PM,246164,19,"GUSA Granules USA, Inc.",,,1
2,Sep 12 2022 12:15PM,246163,10,"Methapharm, Inc.",,,14
3,Sep 12 2022 12:10PM,246161,19,"Innogenix, LLC",,,20
4,Sep 12 2022 12:08PM,246160,10,Emerginnova,,,6
5,Sep 12 2022 12:04PM,246157,19,"GUSA Granules USA, Inc.",,,1
6,Sep 12 2022 11:55AM,246156,16,Sartorius Stedim North America,,,5
7,Sep 12 2022 11:55AM,246155,19,"AdvaGen Pharma, Ltd",,,1
8,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc.",,,21
9,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 12:22PM,246165,10,"Snap Medical Industries, LLC",14,,
1,Sep 12 2022 12:17PM,246164,19,"GUSA Granules USA, Inc.",1,,
2,Sep 12 2022 12:15PM,246163,10,"Methapharm, Inc.",14,,
3,Sep 12 2022 12:10PM,246161,19,"Innogenix, LLC",20,,
4,Sep 12 2022 12:08PM,246160,10,Emerginnova,6,,
5,Sep 12 2022 12:04PM,246157,19,"GUSA Granules USA, Inc.",1,,
6,Sep 12 2022 11:55AM,246156,16,Sartorius Stedim North America,5,,
7,Sep 12 2022 11:55AM,246155,19,"AdvaGen Pharma, Ltd",1,,
8,Sep 12 2022 11:49AM,246154,10,"Methapharm, Inc.",21,,
9,Sep 12 2022 11:46AM,246152,10,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 12:22PM,246165,10,"Snap Medical Industries, LLC",14,,
1,Sep 12 2022 12:17PM,246164,19,"GUSA Granules USA, Inc.",1,,
2,Sep 12 2022 12:15PM,246163,10,"Methapharm, Inc.",14,,
3,Sep 12 2022 12:10PM,246161,19,"Innogenix, LLC",20,,
4,Sep 12 2022 12:08PM,246160,10,Emerginnova,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 12:22PM,246165,10,"Snap Medical Industries, LLC",14.0,NaN,NaN
1,Sep 12 2022 12:17PM,246164,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
2,Sep 12 2022 12:15PM,246163,10,"Methapharm, Inc.",14.0,NaN,NaN
3,Sep 12 2022 12:10PM,246161,19,"Innogenix, LLC",20.0,NaN,NaN
4,Sep 12 2022 12:08PM,246160,10,Emerginnova,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 12 2022 12:22PM,246165,10,"Snap Medical Industries, LLC",14.0,0.0,0.0
1,Sep 12 2022 12:17PM,246164,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
2,Sep 12 2022 12:15PM,246163,10,"Methapharm, Inc.",14.0,0.0,0.0
3,Sep 12 2022 12:10PM,246161,19,"Innogenix, LLC",20.0,0.0,0.0
4,Sep 12 2022 12:08PM,246160,10,Emerginnova,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1969188,64.0,14.0,2.0
15,738377,12.0,22.0,0.0
16,246156,5.0,0.0,0.0
19,1476934,44.0,1.0,0.0
20,492237,16.0,20.0,0.0
21,738351,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1969188,64.0,14.0,2.0
1,15,738377,12.0,22.0,0.0
2,16,246156,5.0,0.0,0.0
3,19,1476934,44.0,1.0,0.0
4,20,492237,16.0,20.0,0.0
5,21,738351,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,64.0,14.0,2.0
1,15,12.0,22.0,0.0
2,16,5.0,0.0,0.0
3,19,44.0,1.0,0.0
4,20,16.0,20.0,0.0
5,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,64.0
1,15,Released,12.0
2,16,Released,5.0
3,19,Released,44.0
4,20,Released,16.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,0.0,0.0,0.0
Executing,14.0,22.0,0.0,1.0,20.0,2.0
Released,64.0,12.0,5.0,44.0,16.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,22.0,0.0,1.0,20.0,2.0
2,Released,64.0,12.0,5.0,44.0,16.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,2.0,0.0,0.0,0.0,0.0,0.0
1,Executing,14.0,22.0,0.0,1.0,20.0,2.0
2,Released,64.0,12.0,5.0,44.0,16.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()